# Importing Libraries and Modules

The aim of this project is to integrate several dealership data in different formats (CSV, JSON, and XML). The car data contain 4 features named "car_model", "year_of_manufacture", "price", and "fuel"

In [177]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

## Set Paths

In [178]:
tmpfile    = "temp.tmp"               # file used to store all extracted data
logfile    = "logfile.txt"            # all event logs will be stored in this file
targetfile = "transformed_data.csv"   # file where transformed data is stored

## Extract

### CSV Extract Function

In [179]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

### JSON Extract Function

In [180]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

### XML Extract Function

In [181]:
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["car_model", "year_of_manufacture", "price", "fuel"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        model = person.find("car_model").text
        year = person.find("year_of_manufacture").text
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        dataframe = dataframe.append({"car_model":model, "year_of_manufacture":year, "price":price, "fuel":fuel}, ignore_index=True)
    return dataframe

### Extract Function

In [182]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

## Transform

The transform function rounds the prices to 2 dp

In [183]:
def transform(data):
        #round off to two decimals(one pound is 0.45359237 kilograms)
        data['price'] = round(data.price,2)
        return data

## Loading

In [184]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile) 

## Logging

In [185]:
def log(message):
    timestamp_format = '%H:%M:%S-%Y-%h-%d' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running ETL Process

In [186]:
log("ETL Job Started")

## Extract Data

In [187]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data.head()

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.000000,Petrol
1,sx4,2013,7089.552239,Diesel
2,ciaz,2017,10820.895522,Petrol
3,wagon r,2011,4253.731343,Petrol
4,swift,2014,6865.671642,Diesel


## Transform Data

In [188]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")

## Load Data

In [189]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [190]:
log("ETL Job Ended")

## Final Data

This is the destination data

In [191]:
transformed_data.head()

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.00,Petrol
1,sx4,2013,7089.55,Diesel
2,ciaz,2017,10820.90,Petrol
3,wagon r,2011,4253.73,Petrol
4,swift,2014,6865.67,Diesel
